# LangChain Basics

In [6]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip show langchain

Name: langchainNote: you may need to restart the kernel to use updated packages.

Version: 0.0.335
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\mramchandani\.conda\envs\stbook\lib\site-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
!more requirements.txt

openai<1
langchain
pinecone-client
python-dotenv
tiktoken


### Python-dotenv

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

In [4]:
load_dotenv(find_dotenv(), override=True)

True

## LLM Models (Wrappers): GPT-3

In [17]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [20]:
prompt='Explain quantum mechanics in one sentence'
output = llm(prompt)
print(output)



Quantum mechanics is a physical theory describing the behavior of matter and energy at the atomic and subatomic level.


In [22]:
print(llm.get_num_tokens(prompt))

7


In [29]:
output = llm.generate(['... is the capital of France.', 'What is the formula for the area of a circle?'])
print([x[0].text for x in output.generations])

['\n\nParis.', '\n\nArea of a circle = π * r2, where r is the radius of the circle.']


In [30]:
print(len(output.generations))

2


In [36]:
output = llm.generate(["Write an original tagline for a burger restaurant"] * 3)

In [37]:
output.generations

[[Generation(text='\n\n"Taste the Juiciness of Our Burgers - We Serve Fresh Flavor!"', generation_info={'finish_reason': 'stop', 'logprobs': None})],
 [Generation(text='\n\n"Taste the Difference - Delicious Burgers at Our Place!"', generation_info={'finish_reason': 'stop', 'logprobs': None})],
 [Generation(text='\n\n"A Burger So Good, You\'ll Think It\'s Magic!"', generation_info={'finish_reason': 'stop', 'logprobs': None})]]

## ChatModels: GPT-3.5-Turbo and GPT-4

In [38]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

In [41]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content=prompt),
]

In [42]:
output = chat(messages)

In [43]:
output

AIMessage(content='Quantenmechanik beschreibt das Verhalten von Teilchen auf subatomarer Ebene und ermöglicht es uns, ihre Eigenschaften und Zustände zu verstehen.')

### Prompt Templates

In [44]:
from langchain import PromptTemplate

In [48]:
template = '''
You are an experienced virologist. Write a few sentences about the following {virus} in {language}.
'''
prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)
print(prompt)

input_variables=['language', 'virus'] template='\nYou are an experienced virologist. Write a few sentences about the following {virus} in {language}.\n'


In [51]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)

In [54]:
output = llm(prompt.format(virus='ebola', language='english'))

In [55]:
print(output)


Ebola is a highly contagious and often fatal virus that causes a severe hemorrhagic fever in humans and other primates. It is transmitted through direct contact with bodily fluids or contaminated objects. Symptoms of Ebola may include fever, headache, fatigue, muscle pain, sore throat, and abdominal pain. In some cases, patients may experience bleeding from the eyes, mouth, and nose. Treatment focuses on providing supportive care and addressing complications as they arise.


In [56]:
output = llm(prompt.format(virus='hiv', language='english'))

In [57]:
print(output)


Human Immunodeficiency Virus (HIV) is a retrovirus that causes Acquired Immunodeficiency Syndrome (AIDS). It is transmitted through contact with bodily fluids, such as through intercourse, blood transfusions, or sharing of needles. HIV attacks the body's immune system, leaving it unable to fight off other infections and diseases, leading to AIDS. There is currently no cure for HIV, but treatments exist which can slow down the virus' progression and enable people with HIV to live longer, healthier lives.


## Simple Chains

In [62]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

template = '''
You are an experienced virologist. Write a few sentences about the following {virus} in {language}.
'''
prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': 'HSV', 'language': 'english'})

In [63]:
print(output)

HSV, or herpes simplex virus, is a highly contagious virus that primarily affects humans. There are two types of HSV: HSV-1, which usually causes oral herpes, and HSV-2, which typically causes genital herpes. Both types can be transmitted through direct contact with infected individuals or through contact with their bodily fluids. HSV infections can cause painful blisters or sores on the skin, and the virus can remain dormant in the body, periodically reactivating and causing outbreaks. While there is no cure for HSV, antiviral medications can help manage symptoms and reduce the frequency of outbreaks.


## Sequential Chains

In [73]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer. Write a function that implments the concept of {concept}'''
)

chain1 = LLMChain(llm=llm1, prompt=prompt1)

llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='''Given the Python function {function}, describe it as detailed as possible'''
)

chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

output = overall_chain.run('softmax')



> Entering new SimpleSequentialChain chain...


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
The function named `softmax(x)` takes in a one-dimensional array `x` as input.

`np.exp(x)` calculates the exponential of each element in the array `x`.

`np.sum(np.exp(x), axis=0)` computes the sum of all elements of the exponential of array `x` along axis 0.

Finally, the function returns the result of dividing the exponential of array `x` by the sum of exponential values, giving a normalized array. The normalized array represents the softmax values of the original input array `x`.

> Finished chain.


In [74]:
pip install langchain_experimental -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI

In [9]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(llm=llm, tool=PythonREPLTool(), verbose=True)
agent_executor.run('Calculate the square root of the factorial of 20 and display it with 4 decimal points')



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to calculate the factorial of 20 and then take the square root of that
Action: Python_REPL
Action Input: from math import factorial; print(round(factorial(20)**0.5, 4))
Observation: 1559776268.6285

Thought: I now know the final answer
Final Answer: 1559776268.6285

> Finished chain.


'1559776268.6285'

In [12]:
agent_executor.run("What is the answer to (5.1 ** 7.3)?")



> Entering new AgentExecutor chain...
 I need to use the Python REPL to calculate this
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought: I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


'146306.05007233328'